In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf, col
from pyspark.sql.types import *
import numpy as np
from sklearn.datasets import make_moons, make_blobs
import pandas as pd


In [2]:
spark = SparkSession.builder.appName('itd-anomaly-profile-dome9-entity-dummy-data-generator').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [3]:
ip_list = ['54.85.1.3', '54.85.5.15', '54.85.10.16', '54.85.20.40', '54.85.23.10', '54.85.23.16', '54.85.9.2', '54.85.9.5', '54.85.12.37', '54.85.16.11', '54.85.17.2', '54.85.17.9', '144.160.7.102', '54.85.21.33', '54.85.22.18', '54.85.2.28', '54.85.2.32', '54.85.9.20', '54.85.17.23', '54.85.18.11', '54.85.25.37', '54.91.220.165', '54.85.11.24', '54.85.11.13', '54.85.2.10', '54.85.19.28', '54.85.20.25', '54.85.21.29', '54.85.5.12', '54.85.12.2', '54.85.16.35', '54.85.22.36', '54.85.3.16', '54.85.12.19', '54.85.22.16', '144.160.99.97', '54.85.2.15', '54.85.3.27', '54.85.20.5', '54.85.21.1', '54.85.24.36', '54.85.8.14', '54.85.22.42', '144.160.227.93', '54.85.23.18', '54.85.25.23', '54.85.12.32', '54.85.13.38', '54.85.5.27', '54.85.7.27', '54.85.12.5', '54.85.25.42', '54.85.2.25', '54.85.10.6', '54.85.12.35', '54.85.14.13', '54.85.15.13', '54.85.19.37', '54.85.20.3', '54.85.23.30', '54.85.115.19', '54.85.3.9', '54.85.3.34', '54.152.216.206', '54.85.12.44', '54.85.21.8', '54.85.3.11', '54.85.11.4', '54.85.12.49', '54.85.13.2', '54.85.18.12', '54.85.5.21', '54.85.13.45', '54.85.16.3', '54.85.16.10', '54.85.19.32', '144.160.7.103', '144.161.1.7', '54.85.2.26', '54.85.2.27', '54.197.95.25', '54.85.8.5', '54.85.10.5', '54.85.11.3', '54.85.24.35', '144.161.1.17', '54.85.2.1', '54.85.13.34', '54.85.15.5', '54.85.16.22', '34.205.18.116', '54.85.24.4', '54.85.25.1', '144.160.29.96', '54.85.1.5', '54.85.5.26', '54.85.11.12', '54.85.12.43', '54.85.23.12', '54.85.3.10', '54.85.9.15', '54.91.220.164', '54.85.10.2', '54.85.16.5', '54.85.18.2', '54.85.19.30', '54.85.22.40', '54.85.115.17', '54.85.3.6', '54.85.6.26', '54.85.8.24', '54.85.9.11', '54.85.14.44', '54.85.23.7', '54.85.25.33', '54.85.4.31', '54.85.5.10', '54.85.17.1', '54.85.19.35', '54.85.22.1', '54.85.22.44', '54.85.11.28', '54.85.22.43', '54.85.23.6', '54.85.1.10', '54.85.115.18', '54.85.13.22', '54.85.6.19', '54.85.11.20', '54.152.216.207', '54.85.13.32', '54.85.21.38', '54.85.25.7', '54.85.3.18', '54.85.8.17', '54.85.17.5', '54.85.19.7', '54.85.21.26', '54.85.9.4', '54.85.11.6', '54.85.11.19', '54.85.22.31', '54.85.23.21', '54.85.7.23', '54.85.8.3', '54.85.20.24', '54.85.2.23', '54.85.20.43', '54.85.24.21', '54.85.115.11', '54.197.97.10', '54.85.5.2', '54.85.8.21', '18.233.98.30', '54.85.21.9', '144.160.99.95', '54.85.4.11', '54.85.7.6', '54.85.15.16', '54.85.22.39', '144.161.1.3', '54.85.4.25', '54.85.7.24', '54.85.15.11', '54.85.19.19', '54.85.116.40', '54.85.5.9', '54.85.9.9', '54.90.128.124', '54.85.13.21', '54.85.20.9', '54.85.20.38', '54.85.24.31', '54.85.25.11', '54.85.8.11', '54.85.18.14', '54.85.4.32', '54.197.95.29', '54.91.220.159', '54.85.21.45', '54.85.25.19', '54.85.25.39', '54.85.7.19', '54.85.21.19', '54.85.24.5', '54.90.128.126', '54.85.16.31', '54.85.19.20', '54.85.18.15', '54.85.3.19', '54.85.5.28', '54.85.7.4', '54.85.7.35', '54.85.14.40', '54.85.15.3', '54.85.17.19', '54.85.19.31', '144.160.6.103', '54.85.3.1', '54.85.6.27', '54.85.14.24', '54.85.16.30', '54.85.22.5', '54.85.22.6', '54.85.23.5', '54.85.25.14', '54.85.19.38', '18.233.98.31', '54.85.16.12', '54.85.17.24', '54.85.22.7', '54.85.23.33', '144.161.1.14', '54.85.4.2', '54.152.216.208', '54.85.17.22', '54.85.1.17', '54.197.197.47', '54.85.8.9', '54.85.11.5', '54.85.22.8', '54.85.3.17', '54.85.10.7', '54.85.13.7', '54.85.13.8', '54.85.14.14', '54.85.21.3', '54.85.13.9', '54.85.23.36', '54.85.10.11', '54.85.14.16', '54.85.22.33', '54.85.11.16', '54.85.17.32', '54.85.18.18', '54.85.20.12', '54.85.4.29', '54.85.7.21', '54.85.11.14', '144.160.5.102', '54.85.19.29', '54.85.23.32', '54.85.24.19', '54.85.2.22', '54.85.3.29', '54.85.6.21', '54.85.10.3', '54.85.14.22', '54.85.17.13', '54.85.20.29', '54.85.8.23', '54.85.10.12', '54.85.21.27', '54.85.4.12', '54.85.5.19', '54.85.6.17', '54.85.13.20', '54.197.97.16', '54.85.12.6', '54.85.20.33', '54.85.16.6', '54.85.21.39', '54.85.21.28', '54.85.22.11', '54.85.3.32', '54.85.12.33', '54.85.13.36', '54.85.17.12', '54.85.20.2', '54.85.23.2', '54.85.4.16', '54.85.5.25', '54.85.14.35', '54.85.23.35', '54.85.24.9', '144.161.1.11', '54.85.4.3', 'sqs12.amazonaws.com', '54.85.8.10', '54.91.220.162', '54.85.12.22', '54.85.18.4', '54.85.20.14', '54.85.20.18', '144.160.98.95', '54.85.25.21', '54.85.2.12', '54.85.8.30', '54.85.13.37', '54.85.13.29', '54.85.14.37', '54.85.5.14', '54.85.13.44', '54.85.16.27', '54.85.17.10', '54.85.24.12', '54.197.97.15', '54.85.5.24', '54.85.7.36', '54.85.12.27', '54.85.13.3', '54.85.15.2', '54.85.16.17', '144.160.5.103', '54.85.23.4', '54.85.25.40', '54.85.1.2', '54.85.11.31', '54.85.13.23', '54.85.14.1', '54.85.21.18', '144.160.99.98', '54.85.3.14', '54.85.8.8', '54.85.9.10', '54.85.25.24', '144.160.199.95', '54.85.8.19', '54.85.17.15', '54.85.2.4', '54.85.3.24', '54.85.7.30', '54.85.8.25', '18.233.98.35', '54.85.20.10', '54.85.24.20', '54.197.95.28', '54.90.128.125', '54.85.12.28', '54.85.14.17', '54.85.22.26', '54.85.1.1', '54.85.11.22', '54.174.105.31', '144.160.6.102', '144.160.7.101', '54.85.1.15', '54.85.2.3', '54.85.1.19', '54.85.115.16', '18.233.98.32', '54.85.14.11', '54.85.17.29', '54.85.18.19', '54.85.24.29', '54.85.4.19', '54.197.95.27', '54.85.13.27', '54.85.17.8', '54.85.17.28', '54.85.18.22', '54.85.19.12', '54.85.1.6', '54.85.2.8', 'sqs1.amazonaws.com', '54.85.8.1', '54.85.8.13', '54.85.13.5', '54.85.15.6', '54.85.2.17', '54.85.6.20', '54.85.11.11', '54.85.11.15', '54.85.11.30', '54.85.2.9', '54.85.8.16', '54.85.21.36', '54.85.22.38', '54.85.24.11', '54.85.1.9', '54.85.4.1', '54.85.13.33', '54.85.14.30', '54.85.19.13', '54.85.5.8', '54.85.7.2', '54.85.9.18', '54.85.13.41', '18.233.98.34', '54.85.14.20', '34.205.18.114', '54.85.25.6', '54.85.12.31', '54.85.15.8', '54.85.22.20', '54.85.25.44', '144.160.20.91', '54.85.4.10', '54.197.97.18', '34.205.17.115', '54.85.22.21', '54.85.24.28', '144.160.31.96', '54.85.7.7', '54.85.12.13', '54.85.18.5', '54.85.18.28', '144.160.8.103', '144.160.21.91', '54.85.2.13', 'firehose2.amazonaws.com', '54.85.13.39', '54.85.24.15', '54.85.25.10', '54.85.3.23', '54.85.10.21', '54.85.14.36', '54.85.14.39', '54.85.18.1', '54.85.18.10', '54.85.18.27', '54.85.17.26', '54.85.2.35', '54.85.116.41', '144.160.18.91', '54.85.2.14', 'AWS Internal', '54.85.5.1', '54.85.7.14', '54.85.12.17', '18.233.99.237', '54.85.25.9', '54.197.197.46', '54.197.97.11', '54.85.8.12', '54.85.12.38', '54.85.15.17', '54.85.20.13', '54.85.24.3', '54.85.24.33', '144.160.99.96', '54.85.5.22', '54.85.7.1', '54.85.21.40', '54.85.23.39', '54.85.2.11', '54.85.2.21', '54.85.17.11', '54.85.21.16', '54.85.9.21', '54.85.12.21', '54.85.19.24', '54.85.22.12', '54.85.4.8', '54.85.6.22', '54.85.13.14', '54.85.19.36', '54.85.4.14', '54.197.97.13', '54.85.7.25', '54.85.8.29', '54.91.220.161', '54.85.10.13', '54.85.11.7', '54.85.12.48', '54.85.16.1', '54.85.22.27', '54.85.6.9', '54.85.21.11', '54.85.25.36', '54.85.8.7', '54.85.16.7', '54.85.115.13', '54.85.17.20', '54.85.21.43', '54.85.4.28', '54.85.6.18', '54.85.10.4', '54.85.12.30', '54.85.17.4', '54.85.19.4', '54.85.23.24', '54.85.23.38', '54.85.25.17', '54.85.2.37', '54.85.5.18', '54.85.13.16', '54.85.15.19', '54.85.17.7', '54.85.25.31', '54.85.25.46', '54.85.1.13', '54.85.6.2', '54.85.6.7', '54.85.7.12', '54.85.7.28', '54.85.17.21', '54.85.18.7', '54.85.22.41', '54.85.25.3', '54.85.3.15', '54.85.4.17', '54.197.97.12', '18.233.98.37', '54.85.14.42', '54.85.115.10', '54.85.115.12', '54.85.12.15', '54.85.23.1', '54.85.24.42', '54.85.3.21', '54.85.23.17', '54.85.6.8', 'sqs13.amazonaws.com', '54.85.14.34', '54.85.14.23', '54.85.21.4', '54.85.24.32', '144.160.227.94', '144.160.198.95', '54.85.1.8', '54.85.10.22', '18.233.98.33', '54.85.18.8', '54.85.24.38', '54.85.25.27', '54.85.6.6', '54.85.10.18', '54.85.14.10', '54.85.14.29', '54.85.25.43', '54.85.10.17', '54.85.13.12', '54.85.14.43', '54.85.19.1', '54.85.22.14', '54.85.24.37', '54.85.25.35', '144.161.1.13', '54.197.97.17', '54.91.220.167', '54.85.9.22', '54.85.12.50', '54.85.16.21', '54.85.12.42', '54.85.19.17', '54.85.2.19', '54.85.4.9', '54.85.4.18', '54.85.16.9', '54.85.19.15', '54.85.2.34', '54.85.3.31', '54.85.4.6', '54.85.5.31', 'sqs2.amazonaws.com', '54.85.16.32', '54.85.21.32', '54.85.3.30', '54.85.7.39', '54.85.2.20', '54.85.10.1', '54.85.14.7', '54.85.18.23', '54.85.19.34', '54.85.20.4', '54.85.25.2', '54.85.115.15', '54.174.105.32', '54.85.20.1', '54.85.20.37', '54.85.21.2', '54.85.1.4', '54.85.7.22', '54.85.9.12', '54.85.16.8', '54.85.20.20', '54.85.22.35', '54.85.24.6', '54.85.6.24', '54.197.95.22', '54.197.95.23', '54.91.220.163', '54.85.13.19', '54.85.14.27', '54.85.15.10', '54.85.15.12', '54.85.16.13', '54.85.18.20', '54.85.18.24', '54.85.21.31', '144.160.99.94', '54.85.5.29', '54.85.9.8', '54.85.13.26', '54.85.14.28', '54.85.16.2', '54.85.17.6', '54.85.21.14', '144.160.30.96', '54.85.116.42', '54.85.14.8', '144.160.98.93', '54.85.22.24', '54.85.23.29', '54.85.25.32', '144.161.1.9', '54.85.7.34', '54.85.9.13', '54.85.13.13', '54.85.13.40', '54.85.14.46', '54.85.1.11', '54.197.97.19', '54.85.7.10', '54.85.23.20', '144.160.200.95', '54.85.5.3', '54.85.15.4', '54.85.16.25', '54.85.24.17', '54.85.5.30', '54.197.95.20', '54.85.12.20', '54.85.20.39', '54.85.21.42', '54.85.23.23', '54.85.25.16', '54.85.4.13', '54.85.6.4', '54.85.6.14', '54.85.7.31', '54.85.19.40', '54.85.20.17', '54.85.22.23', '54.85.23.3', '54.85.12.45', '54.91.18.82', '54.85.19.5', '54.85.21.24', '54.85.22.30', '54.85.24.7', '54.85.3.7', '54.85.8.20', '54.85.10.20', '54.85.14.31', '54.85.25.38', '144.160.226.92', '54.85.6.5', '54.85.19.11', '54.85.19.41', '54.85.2.33', '54.197.95.21', '54.85.13.10', '144.160.8.102', '54.85.23.13', '144.160.228.94', '18.233.98.38', '54.85.25.20', '144.161.1.5', '54.85.6.23', '54.85.10.26', '54.85.17.18', '54.85.20.16', '54.85.22.13', '54.85.3.33', '54.85.5.23', '54.85.8.26', '54.85.10.24', '54.85.11.23', '54.85.12.8', '54.85.19.39', '54.85.21.17', '54.85.2.5', '54.85.4.26', '54.85.14.6', '54.85.17.25', '54.85.14.47', '34.205.17.116', '54.85.20.27', '54.85.4.15', '54.85.7.8', '54.85.12.12', '54.85.14.21', '54.85.1.16', '54.85.14.5', '54.85.16.34', '54.85.20.23', '54.85.5.17', '54.85.6.25', '54.85.10.19', '54.85.11.1', '54.85.12.23', '54.85.13.25', '18.233.99.235', '54.85.25.5', '54.85.1.7', '54.85.13.35', '54.85.13.47', '54.85.19.10', '54.85.20.30', '54.85.22.29', '54.85.24.30', '144.161.1.12', '54.85.7.20', '54.85.12.4', '54.85.13.1', '54.85.13.11', '54.85.20.42', '54.85.23.8', '54.85.10.27', '54.85.16.16', '54.91.18.81', '54.85.21.23', '54.85.24.25', '144.161.1.6', '54.85.9.17', '54.85.12.34', '54.85.16.15', '54.85.16.23', '54.85.17.33', '54.85.19.21', '54.85.25.26', '144.160.27.96', '54.85.7.3', '54.85.16.14', '54.85.3.25', '54.85.7.16', '54.85.8.27', '54.85.19.18', '54.85.25.18', '54.85.3.5', '54.85.8.22', '54.85.11.29', '54.85.13.30', '144.161.1.10', '54.85.4.24', '54.85.21.35', '144.160.228.93', '54.85.7.9', '54.85.12.24', '54.85.17.30', '144.160.28.96', '54.85.6.28', '54.85.7.11', '54.85.13.43', '54.85.15.18', '54.85.17.17', '54.85.21.44', '54.85.25.28', '54.85.12.10', '34.205.17.114', '54.85.18.17', '54.85.18.16', '54.85.18.3', '54.85.18.26', '54.85.20.32', '54.85.20.36', '54.85.21.30', '54.85.24.13', '54.85.24.41', '54.85.3.26', '54.85.11.9', '54.85.12.3', '54.85.14.48', '54.85.22.19', '54.85.23.27', '54.85.9.23', '54.85.12.1', '54.85.19.8', '144.160.9.101', '54.85.24.39', '54.85.25.34', '144.161.1.1', '54.85.10.9', '54.85.11.8', '54.85.14.15', '144.160.98.92', '54.85.11.26', 'firehose.amazonaws.com', '18.233.98.39', '54.85.22.37', '54.85.4.21', '54.85.12.41', '18.233.98.36', '54.85.17.3', '54.85.21.37', '54.85.22.2', '54.85.12.14', '54.85.12.39', '54.85.13.17', '54.85.21.20', '54.85.21.21', '54.85.8.4', '54.85.13.24', '54.85.14.19', '54.85.21.6', '54.85.5.4', '54.85.7.38', '54.85.10.10', '54.85.12.9', '144.160.98.91', '144.160.226.93', '54.85.7.17', '54.85.11.21', '54.85.22.15', '54.85.23.14', '54.85.12.11', '54.85.13.18', '54.85.24.8', '54.85.1.12', '54.85.19.2', '54.85.23.25', '54.85.24.10', '54.85.7.5', '54.85.14.9', '54.85.15.7', '54.85.24.1', '54.85.24.34', '54.85.12.25', '54.85.16.4', '144.160.5.101', '54.85.25.45', '54.85.3.2', '54.85.12.26', '54.85.12.46', '54.85.14.26', '54.85.22.17', '144.160.202.95', '54.85.3.4', '54.85.7.37', '54.85.10.8', '54.85.19.26', '54.85.20.21', '54.85.20.19', '54.85.23.37', '54.85.9.6', '54.85.20.26', '54.85.23.22', '144.160.98.96', '54.85.7.33', '54.85.19.23', '54.85.20.6', '54.85.20.35', '54.85.25.30', '54.85.6.15', '54.85.8.15', '54.85.11.10', '54.85.21.22', '54.85.22.25', '(empty)', '54.85.2.18', '54.85.2.31', '54.85.3.22', '54.85.11.18', '54.85.13.15', '54.85.17.14', '54.85.18.9', '54.85.21.5', '54.85.22.10', '54.85.23.28', '54.85.24.16', 'sqs.amazonaws.com', '54.85.7.15', '54.85.14.45', '54.85.22.32', '54.85.6.1', '54.85.6.29', '54.197.95.24', '54.85.16.33', '54.85.23.34', '144.160.201.95', '144.160.22.91', '54.85.15.14', '54.85.18.25', '144.161.1.4', '144.161.1.18', '54.85.6.11', '54.85.6.16', '54.85.13.28', '54.85.17.27', '54.85.18.21', '54.85.24.43', '54.85.7.13', '54.85.16.36', '54.85.19.27', '54.85.19.22', '54.85.20.22', '54.85.21.7', '54.85.21.34', '54.85.24.2', '54.85.2.16', '54.85.3.3', '54.85.8.6', '54.85.9.7', '54.85.12.36', '54.85.12.47', '54.85.13.42', '54.85.16.24', '54.85.25.12', '54.85.25.48', '144.161.1.2', '54.85.5.6', '54.85.13.46', '54.85.19.14', '54.85.23.41', '54.85.25.4', '54.197.97.14', '54.85.9.16', '54.85.13.31', '18.233.99.236', '54.85.14.3', '54.85.15.9', '54.85.2.29', '54.85.7.26', '54.85.14.33', '54.85.16.18', '144.160.8.101', '54.85.23.9', '144.160.226.91', '54.85.7.18', '54.197.95.26', '144.160.6.101', '54.85.20.34', '54.85.21.13', '54.85.25.25', '54.85.25.41', '54.85.7.32', '54.85.14.41', '54.85.16.28', '54.85.20.31', '54.85.21.10', '54.85.3.12', '54.85.25.29', '54.85.1.18', '54.85.14.38', '54.85.21.25', '144.160.98.94', '144.160.229.93', '54.85.5.16', '54.85.6.12', '54.85.16.19', '54.85.20.11', '54.85.22.34', '54.85.24.18', '54.85.24.24', '54.85.2.7', '54.85.8.2', '54.85.12.7', '54.85.14.25', '54.85.19.33', '54.85.20.8', '54.85.20.28', '54.85.22.3', '54.85.6.13', '54.85.8.28', '54.85.21.41', '54.85.3.28', '54.85.4.27', '54.85.5.11', '54.85.12.40', '54.85.14.4', '54.85.19.3', '54.85.12.29', '54.85.15.1', '54.85.19.9', '144.160.230.94', '54.85.24.27', '54.85.25.47', '54.85.11.17', '54.85.21.15', '54.85.3.8', '54.85.9.3', '54.85.24.14', '54.85.25.22', '54.85.115.14', '54.85.16.26', '54.85.18.13', '54.85.20.7', '54.85.24.22', '54.85.4.5', '54.91.220.168', '54.85.14.2', '54.85.15.20', '54.85.17.31', '54.85.23.26', '54.85.2.6', '54.85.5.13', '54.85.11.2', '54.85.17.16', '54.85.5.7', 'sqs3.amazonaws.com', '54.85.10.23', '54.85.18.6', '54.85.19.25', '54.85.20.41', '54.85.21.46', '54.85.23.15', '54.85.11.27', '54.85.22.4', '54.85.22.22', '54.85.23.19', '54.85.23.40', '54.85.3.20', '54.85.4.7', '54.85.4.20', '54.85.6.10', '54.85.13.4', '54.85.14.18', '144.161.1.8', '54.85.4.30', '54.85.7.29', '54.85.9.14', '54.85.12.16', '54.85.19.16', '54.85.24.40', '144.161.1.16', '54.85.2.24', '54.85.3.13', '54.91.220.160', '54.85.25.8', '54.85.2.36', '54.85.9.19', '34.205.18.115', 'NULL', '54.85.14.32', '54.85.25.15', '144.161.1.15', '54.85.2.38', '54.85.12.18', '54.85.13.6', '54.85.16.29', '54.91.18.80', '54.85.25.13', '54.85.4.4', '54.85.6.30', '54.85.11.25', '54.85.19.6', '54.85.20.15', '144.160.229.94', '54.85.24.26', '54.85.2.30', '54.85.4.22', '54.85.5.5', '54.85.5.20', '54.85.8.18', '54.85.10.15', '54.85.21.12', '54.85.24.23', '144.160.19.91', '54.85.9.1', '54.91.220.166', 'firehose3.amazonaws.com', '54.85.15.15', '54.174.105.30', '54.85.23.31', '144.160.226.94', '54.85.2.2', '54.85.10.14', '54.85.14.12', '54.85.22.9', '54.85.23.11', '54.85.1.14', '54.197.197.48', '54.85.4.23', '54.85.6.3', '54.85.10.25', '54.85.16.20', '54.85.22.28']
user_list = ['astra-es-remodeller-AstraESRemodellerRole-XL763E96FIJ6', 'astra-aws-workload-svm-re-AstraLambdaGetSvmSummari-1AK8RRAAQWFE2', 'pa0780@att.com', 'astra-aws-workload-svm-re-AstraLambdaGetSvmSummari-1DQH8T365MDSO', 'test2-us-east-2', 'rh227x@att.com', 'test3-us-east-2', 'astra-aws-dome9-es-indexer-AstraD9ESIndexerRole-YTRMHW62F3I2', 'test3-eu-west-2', 'astra-aws-dome9-compliance-SchedulerRole-AX6J80MMTWG3', 'test1-us-west-1', 'system', 'astra-aws-dome9-compliance-AlertForwarderRole-I7MOOGLGX7TF', 'astra-aws-es-optimizer-AstraAWSESOptimizerRole-C6315NNONQ5N', 'astra-prod@list.att.com', 'AWSServiceRoleForApplicationAutoScaling_DynamoDBTable', 'test2-eu-west-2', 'astra-aws-dome9-compliance-RemediatorRole-IHHVMOVMZ3M4', 'astra-aws-dome9-compliance-AlertsFilterRole-184MFWS9LE5W5', 'fred.h.meyer@att.com', 'jo757d@att.com', 'astra-aws-report-schedule-AstraAWSReportSchedulerR-18QL5S508GTI4', 'astra-aws-inspector-summa-AstraAWSInspectorSummary-KFVHD4TZ6T30', 'astra-aws-dome9-compliance-InventoryCollectorRole-TBRHLFCWAR80', 'astra-aws-lumberjack-stormLambdaRole-E6SWOD4Q545X', 'astra-aws-lumberjack-S3deliveryRole-X4BH3EWFPAV9', 'astra-aws-ecomp-workload-AstraAWSEcompWorkloadIns-XTNUOPXS215C', 'sa445m@att.com', 'astra-aws-workload-svm-report-AstraLambdaSvmWorker-152245NACSJ9U', 'astra-aws-dome9-ses-notif-AstraAccountNotifierRole-4QKGWH8C11SH', 'astra-aws-dome9-EventParserRole-GEV2FWPS3VXE', 'astra-aws-report-schedule-AstraAWSReportSchedulerR-11BYJX256H7XQ', 'astra-aws-lumberjack-LogRole-ITF9HWQB81OL', 'test1-eu-west-2', 'test2-us-west-1', 'db521d@att.com', 'dd457p@att.com', 'sr2923@att.com', 'AstraSandBoxCrossAccount', 'astra-aws-lumberjack-firehoseLambdaRole-3Z6JLFKBSOB7', 'astra-aws-workload-inspec-AstraAWSWorkloadInspecto-1QW86VYC05Q7O', 'bm2371@att.com', 'test1-us-east-2', 'astra-aws-dome9-accounts-AccountManagerRole-T4YBZ5METEUW', 'astra-aws-workload-svm-report-AstraLambdaSvmWorker-1T0B4EQTFKATZ', 'NULL', 'astra-aws-dome9-compliance-CloudSupervisorRole-TYQSWD6WM243', 'test3-us-west-1']

In [4]:
# #dome9
# TENANT_NAME = "demo"
# data_source = "dome9"

# n_cat_attributes = 2
# n_num_attributes = 3

# cat_feature_names = ["increase_activity","lateral_movement"]
# num_feature_names = ["logon_count","inbound_conn_count","outbound_conn_count"]

# anomaly_type = "profile"
# centers=2
# cluster_std=[i for i in range(10,10+10*centers,10)]
# outliers_fraction = 0.02

In [5]:
# #windows os
# TENANT_NAME = "itd"
# data_source = "windowsos"

# n_cat_attributes = 2
# n_num_attributes = 2

# cat_feature_names = ["increase_activity","lateral_movement"]
# num_feature_names = ["upload_download_ratio","dns_error_count"]

# anomaly_type = "profile"
# centers=2
# cluster_std=[i for i in range(20,20+20*centers,20)]
# outliers_fraction = 0.02

In [6]:
#wgtraffic
TENANT_NAME = "demo"
data_source = "wgtraffic"

n_cat_attributes = 2
n_num_attributes = 2

cat_feature_names = ["wgcat1","wgcat2"]
num_feature_names = ["wgnum1","wgnum2"]

anomaly_type = "profile"
centers=2
cluster_std=[i for i in range(30,30+30*centers,30)]
outliers_fraction = 0.02

In [7]:
# #msexchange
# TENANT_NAME = "itd"
# data_source = "msexchange"

# n_cat_attributes = 2
# n_num_attributes = 2

# cat_feature_names = ["mscat1","mscat2"]
# num_feature_names = ["msnum1","msnum2"]

# anomaly_type = "profile"
# centers=2
# cluster_std=[i for i in range(40,40+40*centers,40)]
# outliers_fraction = 0.02

In [8]:
def get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names):
    
    n_outliers = int(outliers_fraction * n_samples)
    n_inliers = n_samples - n_outliers
    normal = make_blobs(n_samples=n_inliers, n_features=n_num_attributes, cluster_std=cluster_std,
                       shuffle=False, random_state=42,center_box=center_box,centers=centers)[0].astype(int)
    rng = np.random.RandomState(42)
    anomaly = rng.uniform(low=center_box[0], high=center_box[1],
                           size=(n_outliers, n_num_attributes)).astype(int)
    
    if n_num_attributes<=0:
        num_df=None
    else:
        num_data = np.concatenate([normal, anomaly], axis=0)    
        num_df = pd.DataFrame(num_data,columns=num_feature_names)
    
    if n_cat_attributes<=0:
        cat_df = None
    else:
        cat_data = np.random.choice(["True", "False"], size=(n_samples,n_cat_attributes))
        cat_df = pd.DataFrame(cat_data,columns=cat_feature_names)

    if not num_df is None and not cat_df is None:
        df  = pd.concat([cat_df,num_df],axis=1)
    elif num_df is None:
        df = cat_df
    elif cat_df is None:
        df = num_df
    else:
        raise Exception("oops!!!")

    return df



In [9]:
BASE_PATH = "/Users/tuhinsharma/Documents/sstech/"+TENANT_NAME
ANOMALY_DATA_REPOSITORY = BASE_PATH + "/models_data/data"
USER_PROFILE_DATA_PATH = ANOMALY_DATA_REPOSITORY + "/{data_source}/{entity_type}/{anomaly_type}/{time_window}.json"



# Dummy IP Profile with (n_num_attributes+n_cat_attributes) profile attribute

### hour

In [10]:
center_box=(100,500)
n_samples = len(ip_list)

ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["src_ip"] = ip_list
ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="ip",anomaly_type=anomaly_type,
                                            time_window="hour"),"overwrite")




### day

In [11]:
center_box=(100,1500)
n_samples = len(ip_list)

ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["src_ip"] = ip_list
ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="ip",anomaly_type=anomaly_type,
                                            time_window="day"),"overwrite")





### week

In [12]:
center_box=(1000,15000)
n_samples = len(ip_list)

ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["src_ip"] = ip_list
ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="ip",anomaly_type=anomaly_type,
                                            time_window="week"),"overwrite")





### month

In [13]:
center_box=(10000,150000)
n_samples = len(ip_list)

ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["src_ip"] = ip_list
ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="ip",anomaly_type=anomaly_type,
                                            time_window="month"),"overwrite")





# Dummy User Profile with (n_num_attributes+n_cat_attributes) profile attribute

### hour

In [14]:
center_box=(100,500)
n_samples = len(user_list)

ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["user_name"] = user_list

ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="user",anomaly_type=anomaly_type,
                                            time_window="hour"),"overwrite")





### day

In [15]:
center_box=(100,1500)
n_samples = len(user_list)


ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["user_name"] = user_list

ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="user",anomaly_type=anomaly_type,
                                            time_window="day"),"overwrite")


### week

In [16]:
center_box=(1000,15000)
n_samples = len(user_list)



ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["user_name"] = user_list

ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="user",anomaly_type=anomaly_type,
                                            time_window="week"),"overwrite")



### month

In [17]:
center_box=(10000,150000)
n_samples = len(user_list)



ip_df = get_outlier_blob(n_samples,outliers_fraction,centers,cluster_std,center_box,n_cat_attributes,n_num_attributes,cat_feature_names,num_feature_names)

ip_df["user_name"] = user_list
ip_sdf = spark.createDataFrame(ip_df)
ip_sdf.repartition(1).write.json(USER_PROFILE_DATA_PATH.format\
                                           (data_source=data_source,\
                                            entity_type="user",anomaly_type=anomaly_type,
                                            time_window="month"),"overwrite")
